<img src="images/econ157.png" width="200" />

<h1>ECON 157 Class 29</h1>
<h2>Basic Cost Effectiveness Analysis</h2>

This notebook is adapted from the treatment provided by Fernando Alarid-Escudero, Eva Enns, Gregory Knowlton, and the DARTH Team available at [the Comprehensive R Archive Network (CRAN)](https://cran.r-project.org/web/packages/dampack/vignettes/basic_cea.html), under the vignettes available in the `dampack` package.

<h2>Background</h2>

Cost-effectiveness analysis (CEA) compares the costs and effectiveness of treatments across a set of mutually exclusive strategies. 

The first step is to order strategies by increasing costs. As costs increase, effectiveness should also increase. Any strategy with lower effectiveness but higher costs than another strategy is said to be <b>dominated</b>. A rational decision-maker would never implement a dominated strategy because greater effectiveness could be achieved at lower cost by implementing a different strategy. 

Second, the incremental cost and incremental effectiveness of moving from one strategy to the next (in order of increasing costs) are calculated. The incremental cost-effectiveness ratio (<b>ICER</b>) for each strategy is then its incremental costs divided by its incremental effectiveness: 
$$
ICER_{A,B} = \frac{C_A - C_B}{E_A - E_B}
$$

The ICER represents the cost per unit benefit of “upgrading” to that strategy from the next least costly (and next least effective) strategy. 

<b>Weakly dominated</b> strategies are strategies for which there is a linear combination of two different strategies that dominates the strategy (lower costs and/or higher effectiveness). Weak dominance is also called “extended dominance”. Operationally, weakly dominated strategies can be identified by checking that ICERs increase with increasingly costly (and effective) strategies. If there is a “kink” in the trend, then weak/extended dominance exists. Once weakly dominated strategies are removed (and incremental quantities recalculated), the set of remaining strategies form the efficient frontier and associated ICERs can be interpreted for decision-making.

The dampack function `calculate_icers()` completes all of the calculations and checks described above. It takes as inputs the cost, effectiveness outcome (usually QALYs), and strategy name for each strategy, passed as separate vectors. It outputs a specialized data frame that presents the costs and effectiveness of each strategy and, for non-dominated strategies, the incremental costs, effectiveness, and ICER. Dominated strategies are included at the end of the table with the type of dominance indicated as either strong dominance (D) or extended/weak dominance (ED) in the Status column.

We present the application of `calculate_icers()` in the two examples below.

<h1>Example 1:</h1> 
<h2>CEA using average cost and effectiveness of HIV screening strategies in the U.S.</h2>

From: Paltiel AD, Walensky RP, Schackman BR, Seage GR, Mercincavage LM, Weinstein MC, Freedberg KA. Expanded HIV screening in the United States: effect on clinical outcomes, HIV transmission, and costs. Annals of Internal Medicine. 2006;145(11): 797-806. [https://doi.org/10.7326/0003-4819-145-11-200612050-00004](https://www-acpjournals-org.libproxy.berkeley.edu/doi/10.7326/0003-4819-145-11-200612050-00004).

This example explores the costs, benefits, and cost-effectiveness of different HIV screening frequencies in different populations with different HIV prevalence and incidence. To illustrate the CEA functionality of dampack, we will focus on the results evaluating HIV screening frequencies in a high-risk population (1.0% prevalence, 0.12% annual incidence) and accounting only for patient-level benefits, i.e., ignoring any reduction in secondary HIV transmission. 

The statistics we examine are shown in Table 4, Part II: Population-level analysis; No effect of screening/treatment on transmission scenario.

<h3>Data</h3>

Five strategies are considered, ranked from least to most costly:
* No specific screening recommendation (status quo)
* One-time HIV test
* HIV testing every 5 years
* HIV testing every 3 years (which is more costly)
* HIV test annually

We define a vector of short strategy names for labeling our results in the tables and plots.

In [ ]:
#githubinstall("dampack")
install.packages("dampack")
library(dampack)

Costs for each strategy included the cost of the screening strategy and lifetime downstream medical costs for the population. These are shown in Table 4 of Paltiel et al. as noted above, and they are the present value of costs assuming an annual discount rate of 3%. We store the cost of each strategy in a vector, in the same order as in `v_strat_names`.

In [ ]:
v_hiv_strat_names <- c("status quo", "one time", "5yr", "3yr", "annual")

In [ ]:
v_hiv_costs <- c(26000, 27000, 28020, 28440, 29440)
# These are the "Total costs" row shown in Paltiel et al. Table 4, Part II, 
#  No effect of screening/treatment on transmission scenario

The effectiveness of each strategy was measured in terms of quality-adjusted life <i>months</i> (QALMs) of the population, which captures both length of life and quality of life. We convert to quality-adjusted life-years (QALYs) by dividing by 12.

In [ ]:
v_hiv_qalys <- c(277.25, 277.57, 277.78, 277.83, 277.76) / 12
# These are the "Total survival, QALMs" row shown in Paltiel et al. Table 4, Part II, 
#  No effect of screening/treatment on transmission scenario

<h3>Calculate ICERs</h3>

Using these elements, we then use the `calculate_icers()` function in `dampack` to conduct the cost-effectiveness comparison of the five HIV testing strategies.

In [ ]:
icer_hiv <- calculate_icers(cost=v_hiv_costs, 
                            effect=v_hiv_qalys, 
                            strategies=v_hiv_strat_names)
icer_hiv
# These are the outputs we should see:
#>     Strategy  Cost   Effect Inc_Cost  Inc_Effect      ICER Status
#> 1 status quo 26000 23.10417       NA          NA        NA     ND
#> 2   one time 27000 23.13083     1000 0.026666667  37500.00     ND
#> 3        5yr 28020 23.14833     1020 0.017500000  58285.71     ND
#> 4        3yr 28440 23.15250      420 0.004166667 100800.00     ND
#> 5     annual 29440 23.14667       NA          NA        NA      D

The resulting output `icer_hiv` is an icer object (unique to dampack) to facilitate visualization, but it can also be manipulated like a data frame. The default view is ordered by dominance status (ND = non-dominated, ED = extended/weak dominance, or D = strong dominance), and then ascending by cost. In our example, like in Paltiel et al., we see that the annual screening strategy is strongly dominated, though the ICERs calculated here are slightly different from those in the published article, presumably due to rounding.

The `icer` object can be easily formatted into a publication quality HTML table using the `kableExtra` package.

In [ ]:
library(kableExtra)
library(dplyr)
icer_hiv %>%
  kable() %>%
  kable_styling()
#
# We are going to copy and paste the HTML output from this command into a markdown cell below

<table class="table" style="margin-left: auto; margin-right: auto;">
 <thead>
  <tr>
   <th style="text-align:left;"> Strategy </th>
   <th style="text-align:right;"> Cost </th>
   <th style="text-align:right;"> Effect </th>
   <th style="text-align:right;"> Inc_Cost </th>
   <th style="text-align:right;"> Inc_Effect </th>
   <th style="text-align:right;"> ICER </th>
   <th style="text-align:left;"> Status </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> status quo </td>
   <td style="text-align:right;"> 26000 </td>
   <td style="text-align:right;"> 23.10417 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> one time </td>
   <td style="text-align:right;"> 27000 </td>
   <td style="text-align:right;"> 23.13083 </td>
   <td style="text-align:right;"> 1000 </td>
   <td style="text-align:right;"> 0.0266667 </td>
   <td style="text-align:right;"> 37500.00 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> 5yr </td>
   <td style="text-align:right;"> 28020 </td>
   <td style="text-align:right;"> 23.14833 </td>
   <td style="text-align:right;"> 1020 </td>
   <td style="text-align:right;"> 0.0175000 </td>
   <td style="text-align:right;"> 58285.71 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> 3yr </td>
   <td style="text-align:right;"> 28440 </td>
   <td style="text-align:right;"> 23.15250 </td>
   <td style="text-align:right;"> 420 </td>
   <td style="text-align:right;"> 0.0041667 </td>
   <td style="text-align:right;"> 100800.00 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> annual </td>
   <td style="text-align:right;"> 29440 </td>
   <td style="text-align:right;"> 23.14667 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> D </td>
  </tr>
</tbody>
</table>

<h3>Plot CEA results</h3>

The results contained in icer_hiv can be visualized in the cost-effectiveness plane using the built-in `plot()` function, which has its own method for the icer object class.

In [ ]:
plot(icer_hiv)

In the plot, the points on the efficient frontier (consisting of all non-dominated strategies) are connected with a solid line. By default, only strategies on the efficient frontier are labeled. However, this can be changed by setting `label="all"`. There are a number of built-in options for customizing the cost-effectiveness plot. To see a full listing, type `?plot.icers` in the console. 

Also, the plot of an icer object is a ggplot object, so we can add (+) any of the normal ggplot adjustments to the plot.

Plot with all strategies labeled:

In [ ]:
plot(icer_hiv, 
     label="all")

In [ ]:
plot(icer_hiv, 
     label="all") +
  theme_classic() +
  ggtitle("Cost-effectiveness of HIV screening strategies")

<hr>

<h2>Example 2:</h2>

<h3>
    CEA using a probabilistic sensitivity analysis of treatment strategies for <i>Clostridioides difficile</i> infection
    </h3>

From: Rajasingham R, Enns EA, Khoruts A, Vaughn BP. Cost-effectiveness of Treatment Regimens for Clostridioides difficile Infection: An Evaluation of the 2018 Infectious Diseases Society of America Guidelines. Clinical Infectious Diseases. 2020;70(5):754-762. https://doi.org/10.1093/cid/ciz318



In this example, we use a probabilistic sensitivity analysis (PSA) as the basis of our cost-effectiveness calculations, as is now recommended by the Second Panel on Cost-Effectiveness in Health and Medicine (Neumann et al. 2016). For more about PSA and its generation process, please see the PSA vignette by typing `vignette("psa_generation", package = "dampack")` in the console after installing the `dampack` package. 

The PSA dataset in this example was conducted for a model of Clostridioides difficile (C. diff) infection that compared 48 possible treatment strategies, which varied in the treatment regimen used for initial versus recurrent CDI and for different infection severities. For didactic purposes, we have reduced the set of strategies down to the 11 most-relevant strategies; however, in a full CEA, all feasible strategies should be considered, as they are in Rajasingam et al.

Costs in this example include all treatment costs and lifetime downstream medical costs. Strategy effectiveness was measured in terms of quality-adjusted life-expectancy. Outcomes were evaluated for a 67-year-old patient, which is the median age of C. diff infection patients.

<h3>Data</h3>

The <i>C. diff</i> PSA dataset is provided within `dampack` and can be accessed using the `data()` function.

In [ ]:
library(dampack)
data("psa_cdiff")

This creates the object `cdiff_psa`, which is a psa object class (specific to dampack), sharing some properties with data frames. 

To use `calculate_icers()`, we first need to calculate the average cost and average effectiveness for each strategy across the PSA samples. To do this, we use `summary()`, which has its own specific method for psa objects that calculates the mean of each outcome over the PSA samples. For more information, type `?summary.psa` in the console.

In [ ]:
df_cdiff_ce <- summary(psa_cdiff)
head(df_cdiff_ce)
#>   Strategy meanCost meanEffect
#> 1       s3 57336.01   12.93996
#> 2      s27 57541.25   13.01406
#> 3      s33 57642.26   13.03891
#> 4      s31 57934.07   13.09663
#> 5      s43 58072.11   13.11286
#> 6      s44 58665.78   13.12833

Here, strategies are just named with a number (e.g., “s3” or “s39”). The specifications of each strategy can be found in Rajasingam et al.

<h3>Calculate ICERs</h3>

The `df_cdiff_ce` object is a data frame containing the mean cost and mean effectiveness for each of our 11 strategies. We pass the columns of `df_cdiff_ce` to the `calculate_icers()` function to conduct our CEA comparisons.

In [ ]:
icer_cdiff <- calculate_icers(cost = df_cdiff_ce$meanCost,
                              effect = df_cdiff_ce$meanEffect,
                              strategies = df_cdiff_ce$Strategy)
icer_cdiff %>%
  kable() %>%
  kable_styling()

<table class="table" style="margin-left: auto; margin-right: auto;">
 <thead>
  <tr>
   <th style="text-align:left;"> Strategy </th>
   <th style="text-align:right;"> Cost </th>
   <th style="text-align:right;"> Effect </th>
   <th style="text-align:right;"> Inc_Cost </th>
   <th style="text-align:right;"> Inc_Effect </th>
   <th style="text-align:right;"> ICER </th>
   <th style="text-align:left;"> Status </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> s3 </td>
   <td style="text-align:right;"> 57336.01 </td>
   <td style="text-align:right;"> 12.93996 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s27 </td>
   <td style="text-align:right;"> 57541.25 </td>
   <td style="text-align:right;"> 13.01406 </td>
   <td style="text-align:right;"> 205.2466 </td>
   <td style="text-align:right;"> 0.0741001 </td>
   <td style="text-align:right;"> 2769.855 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s33 </td>
   <td style="text-align:right;"> 57642.26 </td>
   <td style="text-align:right;"> 13.03891 </td>
   <td style="text-align:right;"> 101.0061 </td>
   <td style="text-align:right;"> 0.0248476 </td>
   <td style="text-align:right;"> 4065.031 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s31 </td>
   <td style="text-align:right;"> 57934.07 </td>
   <td style="text-align:right;"> 13.09663 </td>
   <td style="text-align:right;"> 291.8156 </td>
   <td style="text-align:right;"> 0.0577142 </td>
   <td style="text-align:right;"> 5056.222 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s43 </td>
   <td style="text-align:right;"> 58072.11 </td>
   <td style="text-align:right;"> 13.11286 </td>
   <td style="text-align:right;"> 138.0394 </td>
   <td style="text-align:right;"> 0.0162319 </td>
   <td style="text-align:right;"> 8504.216 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s44 </td>
   <td style="text-align:right;"> 58665.78 </td>
   <td style="text-align:right;"> 13.12833 </td>
   <td style="text-align:right;"> 593.6686 </td>
   <td style="text-align:right;"> 0.0154752 </td>
   <td style="text-align:right;"> 38362.652 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s39 </td>
   <td style="text-align:right;"> 57814.65 </td>
   <td style="text-align:right;"> 13.04628 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> ED </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s4 </td>
   <td style="text-align:right;"> 57887.48 </td>
   <td style="text-align:right;"> 12.99707 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> D </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s13 </td>
   <td style="text-align:right;"> 58018.63 </td>
   <td style="text-align:right;"> 13.06504 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> D </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s37 </td>
   <td style="text-align:right;"> 58081.79 </td>
   <td style="text-align:right;"> 13.10297 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> D </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s20 </td>
   <td style="text-align:right;"> 58634.20 </td>
   <td style="text-align:right;"> 13.11006 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> D </td>
  </tr>
</tbody>
</table>

In this example, 5 of the 11 strategies are dominated. Most are strongly dominated (denoted by “D”), while one is dominated through extended/weak dominance (denoted “ED”). When many dominated strategies are present in an analysis, it may be desirable to completely remove them from the CEA results table. This can be done by filtering by the `Status` column to include only non-dominated strategies.

In [ ]:
icer_cdiff %>%
  filter(Status == "ND")%>%
  kable() %>%
  kable_styling()

<table class="table" style="margin-left: auto; margin-right: auto;">
 <thead>
  <tr>
   <th style="text-align:left;"> Strategy </th>
   <th style="text-align:right;"> Cost </th>
   <th style="text-align:right;"> Effect </th>
   <th style="text-align:right;"> Inc_Cost </th>
   <th style="text-align:right;"> Inc_Effect </th>
   <th style="text-align:right;"> ICER </th>
   <th style="text-align:left;"> Status </th>
  </tr>
 </thead>
<tbody>
  <tr>
   <td style="text-align:left;"> s3 </td>
   <td style="text-align:right;"> 57336.01 </td>
   <td style="text-align:right;"> 12.93996 </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:right;"> NA </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s27 </td>
   <td style="text-align:right;"> 57541.25 </td>
   <td style="text-align:right;"> 13.01406 </td>
   <td style="text-align:right;"> 205.2466 </td>
   <td style="text-align:right;"> 0.0741001 </td>
   <td style="text-align:right;"> 2769.855 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s33 </td>
   <td style="text-align:right;"> 57642.26 </td>
   <td style="text-align:right;"> 13.03891 </td>
   <td style="text-align:right;"> 101.0061 </td>
   <td style="text-align:right;"> 0.0248476 </td>
   <td style="text-align:right;"> 4065.031 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s31 </td>
   <td style="text-align:right;"> 57934.07 </td>
   <td style="text-align:right;"> 13.09663 </td>
   <td style="text-align:right;"> 291.8156 </td>
   <td style="text-align:right;"> 0.0577142 </td>
   <td style="text-align:right;"> 5056.222 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s43 </td>
   <td style="text-align:right;"> 58072.11 </td>
   <td style="text-align:right;"> 13.11286 </td>
   <td style="text-align:right;"> 138.0394 </td>
   <td style="text-align:right;"> 0.0162319 </td>
   <td style="text-align:right;"> 8504.216 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
  <tr>
   <td style="text-align:left;"> s44 </td>
   <td style="text-align:right;"> 58665.78 </td>
   <td style="text-align:right;"> 13.12833 </td>
   <td style="text-align:right;"> 593.6686 </td>
   <td style="text-align:right;"> 0.0154752 </td>
   <td style="text-align:right;"> 38362.652 </td>
   <td style="text-align:left;"> ND </td>
  </tr>
</tbody>
</table>

<h3>Plot CEA results</h3>

To visualize our results on the cost-effectiveness plane, we can use the plot() function on icer_diff (an icer object).

In [ ]:
plot(icer_cdiff)

In the plot, we can clearly see the one weakly dominated strategy that is more expensive and less beneficial than a linear combination of strategies “s3” and “s31” (a point on the line connecting these two strategies).

Here are some additional plotting options:

In [ ]:
plot(icer_cdiff, label = "all") 
# can lead to a 'busy' plot

In [ ]:
plot(icer_cdiff, plot_frontier_only = TRUE) 
# completely removes dominated strategies from plot

In [ ]:
plot(icer_cdiff, currency = "USD", effect_units = "quality-adjusted life-years") 
# customize axis labels

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>